# Get Data


In [101]:
import numpy as np
import pandas as pd

In [102]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [103]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [104]:
train.shape, test.shape

((1460, 81), (1459, 80))

In [105]:
train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [106]:
test.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


# Cleaning


Some columns can have NaN value, so should overview the columns themselves


In [107]:
train.isna().sum()[train.isna().sum() > 0]

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

## Remove Duplicates


In [108]:
train.drop_duplicates(inplace=True)

## MSSubClass

This is stored as a number, but behaves as a category


In [109]:
train['MSSubClass'] = train['MSSubClass'].astype('object')
test['MSSubClass'] = test['MSSubClass'].astype('object')

## LotFrontage

The NaN values here mostly mean incomplete data, so it will be filled with the median (because it is continuous data)


In [110]:
LotFrontage_median = train['LotFrontage'].median()
train.loc[train['LotFrontage'].isna(), 'LotFrontage'] = LotFrontage_median
test.loc[test['LotFrontage'].isna(), 'LotFrontage'] = LotFrontage_median

## MasVnrType

NaN here means that it does not exist, so MasVnrArea should be 0


In [111]:
train.loc[train['MasVnrType'].isna(), 'MasVnrArea'] = 0
test.loc[test['MasVnrType'].isna(), 'MasVnrArea'] = 0

## Electric

NaN here likely means that is was not documented, so it will be set to the most common one


In [112]:
most_common_electrical = train['Electrical'].mode()[0]
train.loc[train['Electrical'].isna(), 'Electrical'] = most_common_electrical
test.loc[test['Electrical'].isna(), 'Electrical'] = most_common_electrical

## GarageYrBlt

When a house does not have a garage, the year for it is NaN. Giving all of these values some year value would be wrong, so I decided to put these years into bins, where the NaNs will all go inside the same bin


In [113]:
bins = [0, 1900, 1920, 1940, 1960, 1980, 2000, 2020]
labels = ['None', '1900-1919', '1920-1939', '1940-1959', '1960-1979', '1980-1999', '2000-2019']

train['GarageYrBltInt'] = pd.cut(train['GarageYrBlt'].fillna(0), bins=bins, labels=labels, right=False).astype('object')
test['GarageYrBltInt'] = pd.cut(test['GarageYrBlt'].fillna(0), bins=bins, labels=labels, right=False).astype('object')

train.drop(columns=['GarageYrBlt'], inplace=True)
test.drop(columns=['GarageYrBlt'], inplace=True)


## Other Columns

The rest of the columns where it is NaN is actually a valid category, so I will change them to 'None'


In [114]:
train.fillna('None', inplace=True)
test.fillna('None', inplace=True)

/tmp/ipykernel_154433/3694420305.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test.fillna('None', inplace=True)


## Check cleaned


In [115]:
train.isna().sum()[train.isna().sum() > 0]

Series([], dtype: int64)

In [116]:
train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,GarageYrBltInt
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500,2000-2019
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500,1960-1979
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500,2000-2019
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000,1980-1999
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000,2000-2019
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,None,Attchd,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000,1980-1999
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,None,None,None,0,8,2007,WD,Normal,307000,2000-2019
7,8,60,RL,69.0,10382,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,None,None,Shed,350,11,2009,WD,Normal,200000,1960-1979
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,None,None,None,0,4,2008,WD,Abnorml,129900,1920-1939
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,Metal

# Feature Engineering


In [117]:
def check_columns(df):
    cat_cols = df.select_dtypes(include=['object']).columns
    num_cols = df.select_dtypes(exclude=['object']).columns
    
    print(f"Categorical columns: {len(cat_cols)}")
    print(cat_cols)
    print(f"Numerical columns: {len(num_cols)}")
    print(num_cols)

In [118]:
check_columns(train)

Categorical columns: 45
Index(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'GarageYrBltInt'],
      dtype='object')
Numerical columns: 36
Index(['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath',

In [119]:
cat_cols = train.select_dtypes(include=['object']).columns

## Category Encoding


In [120]:
from sklearn.preprocessing import LabelEncoder

train_ref = train.copy()

### Definitions


In [121]:
def one_hot_encoding(df, cat_cols):
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    return df

In [122]:
def label_encoding(df, cat_cols):
    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])
    return df

In [123]:
def target_encoding(df, cat_cols, target_col, df_ref=None):
    if df_ref is None:
        df_ref = df
    for col in cat_cols:
        mean = df_ref.groupby(col)[target_col].mean()
        df[col] = df[col].map(mean)
    return df

In [124]:
def frequency_encoding(df, cat_cols, def_ref=None):
    if df_ref is None:
        df_ref = df
    for col in cat_cols:
        freq = df_ref[col].value_counts(normalize=True)
        df[col] = df[col].map(freq)
    return df

### Encoding

In [125]:
label_encoding(train, cat_cols)
label_encoding(test, cat_cols)
train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,GarageYrBltInt
0,1,5,3,65.0,8450,1,1,3,3,0,4,0,5,2,2,0,5,7,5,2003,2003,1,1,12,13,1,196.0,2,4,2,2,4,3,2,706,6,0,150,856,1,0,1,4,856,854,0,1710,1,0,2,1,3,1,2,8,6,0,3,1,2,2,548,5,5,2,0,61,0,0,0,0,3,4,1,0,2,2008,8,4,208500,5
1,2,0,3,80.0,9600,1,1,3,3,0,2,0,24,1,2,0,2,6,8,1976,1976,1,1,8,8,2,0.0,3,4,1,2,4,1,0,978,6,0,284,1262,1,0,1,4,1262,0,0,1262,0,1,2,0,3,1,3,6,6,1,5,1,2,2,460,5,5,2,298,0,0,0,0,0,3,4,1,0,5,2007,8,4,181500,3
2,3,5,3,68.0,11250,1,1,0,3,0,4,0,5,2,2,0,5,7,5,2001,2002,1,1,12,13,1,162.0,2,4,2,2,4,2,2,486,6,0,434,920,1,0,1,4,920,866,0,1786,1,0,2,1,3,1,2,6,6,1,5,1,2,2,608,5,5,2,0,42,0,0,0,0,3,4,1,0,9,2008,8,4,223500,5
3,4,6,3,60.0,9550,1,1,0,3,0,0,0,6,2,2,0,5,7,5,1915,1970,1,1,13,15,2,0.0,3,4,0,4,1,3,0,216,6,0,540,756,1,2,1,4,961,756,0,1717,1,0,1,0,3,1,2,7,6,1,2,5,3,3,642,5,5,2,0,35,272,0,0,0,3,4,1,0,2,2006,8,0,140000,4
4,5,5,3,84.0,14260,1,1,0,3,0,2,0,15,2,2,0,5,8,5,2000,2000,1,1,12,13,1,350.0,2,4,2,2,4,0,2,655,6,0,490,1145,1,0,1,4,1145,1053,0,2198,1,0,2,1,4,1,2,9,6,1,5,1,2,3,836,5,5,2,192,84,0,0,0,0,3,4,1,0,12,2008,8,4,250000,5
5,6,4,3,85.0,14115,1,1,0,3,0,4,0,11,2,2,0,0,5,5,1993,1995,1,1,12,13,2,0.0,3,4,5,2,4,3,2,732,6,0,64,796,1,0,1,4,796,566,0,1362,1,0,1,1,1,1,3,5,6,0,3,1,3,2,480,5,5,2,40,30,0,320,0,0,3,2,3,700,10,2009,8,4,143000,4
6,7,0,3,75.0,10084,1,1,3,3,0,4,0,21,2,2,0,2,8,5,2004,2005,1,1,12,13,3,186.0,2,4,2,0,4,0,2,1369,6,0,317,1686,1,0,1,4,1694,0,0,1694,1,0,2,0,3,1,2,7,6,1,2,1,2,2,636,5,5,2,255,57,0,0,0,0,3,4,1,0,8,2007,8,4,307000,5
7,8,5,3,69.0,10382,1,1,0,3,0,0,0,14,4,2,0,5,7,6,1973,1973,1,1,6,6,3,240.0,3,4,1,2,4,2,0,859,1,32,216,1107,1,0,1,4,1107,983,0,2090,1,0,2,1,3,1,3,7,6,2,5,1,2,2,484,5,5,2,235,204,228,0,0,0,3,4,3,350,11,2009,8,4,200000,3
8,9,4,4,51.0,6120,1,1,3,3,0,4,0,17,0,2,0,0,7,5,1931,1950,1,1,3,15,2,0.0,3,4,0,4,4,3,6,0,6,0,952,952,1,2,1,1,1022,752,0,1774,0,0,2,0,2,2,3,8,2,2,5,5,3,2,468,1,5,2,90,0,205,0,0,0,3,4,1,0,4,2008,8,0,129900,1
9,10,14,3,50.0,7420,1,1,3,3,0,0,0,3,0,0,1,1,5,6,1939,1950,1,1,8,8,2,0.0,3,4,0,4,4,3,2,851,6,0,140,991,1,0,1,4,1077,0,0,1077,1,0,1,0,2,2,3,5,6,2,5,1,2,1,205,2,5,2,0,4,0,0,0,0,3,4,1,0,1,2008,8,4,118000,1


In [126]:
# target_encoding(train, cat_cols, 'SalePrice')
# target_encoding(test, cat_cols, 'SalePrice', train_ref)
# test.head(10)

In [127]:
check_columns(train)

Categorical columns: 0
Index([], dtype='object')
Numerical columns: 81
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageCars', 'Gar

# Feature Selection


In [128]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [129]:
def correlation_filter(df, target_col, threshold=0.8):
    corr = df.drop(columns=[target_col]).corr().abs()

    to_drop_pairs = []
    for i in range(len(corr.columns)):
        for j in range(i):
            if corr.iloc[i, j] > threshold:
                to_drop_pairs.append((corr.columns[i], corr.columns[j], corr.iloc[i, j]))

    print("Highly correlated pairs:")
    for col1, col2, corr_val in to_drop_pairs:
        print(f"{col1} and {col2}: {corr_val}")
        
    to_drop = set()
    for col1, col2, _ in to_drop_pairs:
        if abs(df[col1].corr(df[target_col])) > abs(df[col2].corr(df[target_col])):
            to_drop.add(col2)
        else:
            to_drop.add(col1)
            
    to_drop = list(to_drop)
    
    print(f"Columns to drop: {to_drop}")
    
    return to_drop

In [130]:
def rfe(df, target_col, n_features_to_select=10):
    x = df.drop(columns=[target_col, 'Id'])
    y = df[target_col]

    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=n_features_to_select, step=1)
    rfe.fit(x, y)

    selected_features = x.columns[rfe.support_].tolist()
    
    print(f"Selected features: {selected_features}")
    
    return selected_features

In [131]:
to_drop = correlation_filter(train, 'SalePrice', 0.8)

Highly correlated pairs:
Exterior2nd and Exterior1st: 0.8541629837325787
1stFlrSF and TotalBsmtSF: 0.8195299750050339
TotRmsAbvGrd and GrLivArea: 0.8254893743088425
GarageArea and GarageCars: 0.882475414281462
PoolQC and PoolArea: 0.8842503024391035
Columns to drop: ['TotRmsAbvGrd', 'Exterior1st', '1stFlrSF', 'PoolArea', 'GarageArea']


In [132]:
filtered_train = train.drop(columns=to_drop)
filtered_test = test.drop(columns=to_drop)

In [133]:
to_stay = rfe(filtered_train, 'SalePrice', 60)

Selected features: ['MSSubClass', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'RoofStyle', 'RoofMatl', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'ScreenPorch', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'GarageYrBltInt']


In [134]:
filtered_train = filtered_train[to_stay + ['SalePrice', 'Id']]
filtered_test = filtered_test[to_stay + ['Id']]

In [135]:
filtered_train.head(10)

,MSSubClass,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,RoofStyle,RoofMatl,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,ScreenPorch,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,GarageYrBltInt,SalePrice,Id
0,5,65.0,1,1,3,3,0,4,0,5,2,2,0,5,7,5,2003,1,1,13,1,2,4,2,2,4,3,2,6,1,0,1,4,0,1710,1,0,2,1,3,1,2,6,0,3,1,2,2,5,5,2,0,3,4,1,2,2008,8,4,5,208500,1
1,0,80.0,1,1,3,3,0,2,0,24,1,2,0,2,6,8,1976,1,1,8,2,3,4,1,2,4,1,0,6,1,0,1,4,0,1262,0,1,2,0,3,1,3,6,1,5,1,2,2,5,5,2,0,3,4,1,5,2007,8,4,3,181500,2
2,5,68.0,1,1,0,3,0,4,0,5,2,2,0,5,7,5,2001,1,1,13,1,2,4,2,2,4,2,2,6,1,0,1,4,0,1786,1,0,2,1,3,1,2,6,1,5,1,2,2,5,5,2,0,3,4,1,9,2008,8,4,5,223500,3
3,6,60.0,1,1,0,3,0,0,0,6,2,2,0,5,7,5,1915,1,1,15,2,3,4,0,4,1,3,0,6,1,2,1,4,0,1717,1,0,1,0,3,1,2,6,1,2,5,3,3,5,5,2,0,3,4,1,2,2006,8,0,4,140000,4
4,5,84.0,1,1,0,3,0,2,0,15,2,2,0,5,8,5,2000,1,1,13,1,2,4,2,2,4,0,2,6,1,0,1,4,0,2198,1,0,2,1,4,1,2,6,1,5,1,2,3,5,5,2,0,3,4,1,12,2008,8,4,5,250000,5
5,4,85.0,1,1,0,3,0,4,0,11,2,2,0,0,5,5,1993,1,1,13,2,3,4,5,2,4,3,2,6,1,0,1,4,0,1362,1,0,1,1,1,1,3,6,0,3,1,3,2,5,5,2,0,3,2,3,10,2009,8,4,4,143000,6
6,0,75.0,1,1,3,3,0,4,0,21,2,2,0,2,8,5,2004,1,1,13,3,2,4,2,0,4,0,2,6,1,0,1,4,0,1694,1,0,2,0,3,1,2,6,1,2,1,2,2,5,5,2,0,3,4,1,8,2007,8,4,5,307000,7
7,5,69.0,1,1,0,3,0,0,0,14,4,2,0,5,7,6,1973,1,1,6,3,3,4,1,2,4,2,0,1,1,0,1,4,0,2090,1,0,2,1,3,1,3,6,2,5,1,2,2,5,5,2,0,3,4,3,11,2009,8,4,3,200000,8
8,4,51.0,1,1,3,3,0,4,0,17,0,2,0,0,7,5,1931,1,1,15,2,3,4,0,4,4,3,6,6,1,2,1,1,0,1774,0,0,2,0,2,2,3,2,2,5,5,3,2,1,5,2,0,3,4,1,4,2008,8,0,1,129900,9
9,14,50.0,1,1,3,3,0,0,0,3,0,0,1,1,5,6,1939,1,1,8,2,3,4,0,4,4,3,2,6,1,0,1,4,0,1077,1,0,1,0,2,2,3,6,2,5,1,2,1,2,5,2,0,3,4,1,1,2008,8,4,1,118000,10


# Training

In [136]:
x_train = filtered_train.drop(columns=['Id', 'SalePrice'])
train_ids = filtered_train['Id']
y_train = filtered_train['SalePrice']

x_test = filtered_test.drop(columns=['Id'])
test_ids = filtered_test['Id']
x_train.shape, y_train.shape, x_test.shape

((1460, 60), (1460,), (1459, 60))

In [137]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

In [138]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

scalers = [
    StandardScaler(),
    MinMaxScaler(),
    RobustScaler(),
    Normalizer(),
    None
]

kfold = KFold(n_splits=15, shuffle=True, random_state=42)

param_grid = [
    {
        'scaler': scalers,
        'regressor': [LinearRegression()],
        'regressor__fit_intercept': [True, False]
    },
    {
        'scaler': scalers,
        'regressor': [Ridge(max_iter=10000)],
        'regressor__alpha': [0.1, 1, 10]
    },
    {
        'scaler': scalers,
        'regressor': [Lasso(max_iter=10000)],
        'regressor__alpha': [0.01, 0.1, 1, 10, 100]
    }
]

In [139]:
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=kfold,
    scoring='neg_mean_squared_error',
    verbose=2,
    return_train_score=True
)

In [140]:
grid_search.fit(x_train, y_train)

Fitting 15 folds for each of 50 candidates, totalling 750 fits
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.1s
[CV] END regressor=LinearRegression(), regressor__fit_intercept=True, scaler=StandardScaler(); total time=   0.0s
[CV] END regressor=Linear

GridSearchCV(cv=KFold(n_splits=15, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('regressor', LinearRegression())]),
             param_grid=[{'regressor': [LinearRegression()],
                          'regressor__fit_intercept': [True, False],
                          'scaler': [StandardScaler(), MinMaxScaler(),
                                     RobustScaler(), Normalizer(), None]},
                         {'regressor': [Ridge(max_iter=10000)],
                          'regressor__alpha': [0.1, 1, 10],
                          'scaler': [StandardScaler(), MinMaxScaler(),
                                     RobustScaler(), Normalizer(), None]},
                         {'regressor': [Lasso(max_iter=10000)],
                          'regressor__alpha': [0.01, 0.1, 1, 10, 100],
                          'scaler': [StandardScaler(), MinMaxScaler(),
                                     RobustScaler(), Normalizer(), None]}],
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [141]:
def get_results(grid_search):
    results = pd.DataFrame(grid_search.cv_results_)
    results = results.sort_values(by='rank_test_score')
    results = results[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]
    results['rmse_test_score'] = np.sqrt(-results['mean_test_score'])
    results['rmse_train_score'] = np.sqrt(-results['mean_train_score'])
    results = results[['params', 'rmse_train_score', 'rmse_test_score', 'rank_test_score']]
    return results

In [142]:
print(get_results(grid_search))

                                               params  rmse_train_score  rmse_test_score  rank_test_score
46  {'regressor': Lasso(max_iter=10000), 'regresso...      32050.646155     34048.314971                1
33  {'regressor': Lasso(max_iter=10000), 'regresso...      32585.183233     34412.700217                2
22  {'regressor': Ridge(max_iter=10000), 'regresso...      31613.093092     34685.541001                3
47  {'regressor': Lasso(max_iter=10000), 'regresso...      31657.320350     34731.985206                4
49  {'regressor': Lasso(max_iter=10000), 'regresso...      31656.215752     34754.663024                5
24  {'regressor': Ridge(max_iter=10000), 'regresso...      31607.313436     34758.113018                6
16  {'regressor': Ridge(max_iter=10000), 'regresso...      31791.289331     34761.914912                7
28  {'regressor': Lasso(max_iter=10000), 'regresso...      32314.410698     34815.531824                8
6   {'regressor': LinearRegression(), 'regress

# Dagshub Upload

In [143]:
import dagshub
import mlflow
from mlflow.models.signature import infer_signature
dagshub.init(repo_owner='Cimbir', repo_name='advanced-regression-techniques', mlflow=True)

Initialized MLflow to track repo "Cimbir/advanced-regression-techniques"

Repository Cimbir/advanced-regression-techniques initialized!

In [ ]:
with mlflow.start_run(run_name='grid_search_label_encoding') as run:
    run_id = run.info.run_id
    
    results = get_results(grid_search)
    
    for i, row in results.iterrows():
        param_key = f"model_{i}"
        param_value = str(row['params'])
        print(row['params'])
        mlflow.log_param(param_key, param_value)
        mlflow.log_metric(f"rmse_train_score_{i}", row['rmse_train_score'])
        mlflow.log_metric(f"rmse_test_score_{i}", row['rmse_test_score'])

    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)
    mlflow.log_params(best_params)
    score = grid_search.best_score_
    score_sqrt_abs = np.sqrt(abs(score))
    print("Best RMSE Score:", score_sqrt_abs)
    mlflow.log_metric("best_score_sqrt_abs", score_sqrt_abs)

    input_example = x_train.iloc[:1]
    signature = infer_signature(input_example, grid_search.predict(input_example))
    
    model_name = "best_model"
    
    mlflow.sklearn.log_model(
        grid_search.best_estimator_, 
        artifact_path=model_name,
        input_example=input_example, 
        signature=signature)
    
    mlflow.register_model(f"runs:/{run_id}/{model_name}", model_name)

    print("Logged to MLflow")